<a id='1'></a>
# Импортируйте пакеты

In [ ]:
import tensorflow.python.keras.backend as K


<a id='4'></a>
# Конфигурация модели

In [ ]:
K.set_learning_phase(0)

In [ ]:
# Input/Output resolution
RESOLUTION = 64  # 64x64, 128x128, 256x256
assert (RESOLUTION % 64) == 0, "RESOLUTION should be 64, 128, 256"

In [ ]:
# Architecture configuration
arch_config = {
    "IMAGE_SHAPE": (RESOLUTION, RESOLUTION, 3),
    "use_self_attn": True,
    "norm": "instancenorm",
    "model_capacity": "standard"
}

<a id='5'></a>

# Определение моделей

In [ ]:
from networks.faceswap_model import FaceswapModel

In [ ]:
model = FaceswapModel(**arch_config)

<a id='6'></a>
# Загрузка весов моделей

In [ ]:
model.load_weights(path="./models")

<a id='12'></a>
# Конвертация видео

In [ ]:
from converter import VideoConverter
from detector.face_detector import MTCNNFaceDetector


In [ ]:
mtcnn_weights_dir = "./mtcnn_weights/"

fd = MTCNNFaceDetector(sess=K.get_session(), model_path=mtcnn_weights_dir)
vc = VideoConverter()

vc.set_face_detector(fd)
vc.set_gan_model(model)

### Конфигурация преобразования видео


- `use_smoothed_bbox`: 
    - Boolean. Следует ли включать сглаженный bbox.
- `use_kalman_filter`: 
    - Boolean. Следует ли включать фильтр Калмана.
- `use_auto_downscaling`:
    - Boolean. Следует ли включать автоматическое уменьшение масштаба при распознавании лиц (для предотвращения ошибки ООМ).
- `bbox_moving_avg_coef`: 
    - Float point в диапазоне [0.0, 1.0]. Коэффициент сглаживания, используемый, когда значение use_kalman_filter равно False.
- `min_face_area`:
    - int x int. Минимальный размер лица. Обнаруженные лица размером меньше min_face_area преобразовываться не будут.
- `IMAGE_SHAPE`:
    - Разрешение ввода/вывода модели GAN.
- `kf_noise_coef`:
    - Float point. Увеличьте в 10 раз, если отслеживание происходит медленно. Снизьте на 1/10 раза, если отслеживание работает нормально, но возникает дрожание.
- `use_color_correction`: 
    - Строка со значением "adain", "adain_xyz", "hist_match", или "none". Применяемый метод цветокоррекции.
- `detec_threshold`: 
    - Float point в диапазоне [0.0, 1.0]. Уменьшите его значение, если лица пропущены. Увеличьте его значение, чтобы уменьшить количество ложных срабатываний.
- `roi_coverage`: 
    - Float point в диапазоне [0.0, 1.0). Центральная область входных изображений, подлежащих обрезке (Рекомендуемый диапазон: 0.85 ~ 0.95)
- `enhance`: 
    - Float point. Коэффициент усиления контраста в области альфа-маски (Рекомендуемый диапазон: 0. ~ 0.4)
- `output_type`: 
    - Формат компоновки выходного видео: 1. [ result ], 2. [ source | result ], 3. [ source | result | mask ]
- `direction`: 
    - Строка со значением "AtoB" или "BtoA". Направление трансформации лица.

In [ ]:
options = {
    # ===== Fixed =====
    "use_smoothed_bbox": True,
    "use_kalman_filter": True,
    "use_auto_downscaling": False,
    "bbox_moving_avg_coef": 0.65,
    "min_face_area": 35 * 35,
    "IMAGE_SHAPE": model.IMAGE_SHAPE,
    # ===== Tunable =====
    "kf_noise_coef": 3e-3,
    "use_color_correction": "hist_match",
    "detec_threshold": 0.7,
    "roi_coverage": 0.9,
    "enhance": 0.,
    "output_type": 3,
    "direction": "AtoB",
}

# Начать преобразование видео

- `input_fn`: 
    - String. Путь к входному видео.
- `output_fn`: 
    - String. Путь к выходному видео.
- `duration`: 
    - `None` или неотрицательный кортеж с плавающей точкой: (start_sec, end_sec). Продолжительность входного видео, подлежащего преобразованию, например, установка "duration=(5, 7,5)` выводит видеоклип продолжительностью 2,5 секунды, соответствующий 5с~7,5с входного видео.


In [ ]:
input_fn = "INPUT_VIDEO.mp4"
output_fn = "OUTPUT_VIDEO.mp4"
duration = None

In [ ]:
vc.convert(input_fn, output_fn, options, duration)